In [ ]:
# 这会将你的Google云端硬盘导入到Colab VM。
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 输入daseCV所在的路径
# 'daseCV' 文件夹包括 '.py', 'classifiers' 和'datasets'文件夹
# 例如 'CV/assignments/assignment1/daseCV/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# 现在直接将Colab VM跟Google云端硬盘建立连接
# 不再将Google云端硬的文件拷贝到Colab VM
# 这样可以实时保存Google云端硬盘的文件
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# 下载 CIFAR-10 数据集到colab
%cd drive/My\ Drive/$FOLDERNAME/daseCV/datasets/
!bash get_datasets.sh
%cd /content

# Network Visualization (TensorFlow)

在该notebook中，我们将探讨使用*图像梯度*生成新图像的方法。

在训练模型时，我们定义了一个损失函数，该函数可以衡量当前模型的好坏程度。 然后，我们使用反向传播算法计算模型参数对损失的梯度，并对模型参数执行梯度下降以最大程度地减小损失。

在这里，我们将做一些稍微不同的事情。 我们使用在ImageNet数据集上经过预训练的卷积神经网络。 我们将使用该模型定义一个损失函数，该函数可以量化我们对当前图像的不满意程度，然后使用反向传播来计算该损失相对于图像像素的梯度。 然后，我们将冻结模型参数固定不变，并对*图像*执行梯度下降，以合成新图像，从而最大程度地减少损失。

在该notebook中，我们将探讨三种图像生成技术：

1. **Saliency Maps**: Saliency maps可以快速判断图像的哪一部分影响了网络的分类决策。
2. **Fooling Images**: 我们可以对输入图像进行扰动，使它在人眼里变化不大，但是会让预训练的网络做出错误分类。
3. **Class Visualization**: 我们可以合成图像使得特定类别的分类最大化。 这可以告诉我们网络在对该类别的图像进行分类时到底在寻找什么。

该notebook使用 **TensorFlow** ; 我们提供了另一个notebook，探讨了PyTorch中的相同概念。 你只需要完成这两个notebook中的一个即可。

In [ ]:
# As usual, a bit of setup
import sys
sys.path.append('../')
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from daseCV.classifiers.squeezenet import SqueezeNet
from daseCV.data_utils import load_tiny_imagenet
from daseCV.image_utils import preprocess_image, deprocess_image
from daseCV.image_utils import SQUEEZENET_MEAN, SQUEEZENET_STD

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Pretrained Model

对于我们所有的图像生成实验，我们将从在ImageNet数据集上经过预训练的卷积神经网络开始。我们可以在这里使用任何模型，在这里我们挑选了SqueezeNet [1]，因为它可获得与AlexNet相当的精度，但参数数量和计算复杂度却大大降低。

使用SqueezeNet而不是AlexNet或VGG或ResNet意味着我们可以轻松地在CPU上执行所有图像生成实验。

我们已经导入 PyTorch SqueezeNet model 到 TensorFlow 中; 从: `daseCV/classifiers/squeezenet.py` 中查看模型结构。

要使用SqueezeNet，你将需要先 **下载权重** ，方法是进入 `daseCV / datasets` 目录并运行 `get_squeezenetnet_tf.sh` 。 请注意，如果你运行了 `get_assignment3_data.sh` ，那么SqueezeNet就已经下载好了。

下载Squeezenet模型后，我们就可以将其加载到新的TensorFlow session中：

[1] Iandola et al, "SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and < 0.5MB model size", arXiv 2016

**NOTE:**  Ignore Tensorflow warnings from the cell below

In [ ]:
SAVE_PATH = None

# Local
#SAVE_PATH = 'daseCV/datasets/squeezenet.ckpt'

# Colab
#SAVE_PATH = '/content/drive/My Drive/{}/{}'.format(FOLDERNAME, 'daseCV/datasets/squeezenet.ckpt')

assert SAVE_PATH is not None, "[!] Choose path to squeezenet.ckpt"

if not os.path.exists(SAVE_PATH + ".index"):
    raise ValueError("You need to download SqueezeNet!")

model = SqueezeNet()
status = model.load_weights(SAVE_PATH)

model.trainable = False

## Load some ImageNet images
我们提供了一些从ImageNet ILSVRC 2012分类数据集的验证集中得到的示例图像。你可以进入 `daseCV/datasets/` 文件夹并运行 `get_imagenet_val.sh` 就可以下载该数据集。

由于它们来自验证集，因此我们的预训练模型在训练过程中看不到这些图像。

运行以下单元格以可视化其中一些图像以及其分类标签。

In [ ]:
from daseCV.data_utils import load_imagenet_val
X_raw, y, class_names = load_imagenet_val(num=5)

plt.figure(figsize=(12, 6))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_raw[i])
    plt.title(class_names[y[i]])
    plt.axis('off')
plt.gcf().tight_layout()

## Preprocess images
预训练模型希望正则化后的输入，因此我们首先通过减去像素平均数并除以像素标准差来对图像进行预处理。

In [ ]:
X = np.array([preprocess_image(img) for img in X_raw])

# Saliency Maps
使用这个经过预训练的模型，我们将按照[2]的3.1节中的描述计算class saliency maps。

**saliency map**告诉我们图像中每个像素影响该图像分类得分的程度。 为了得到它，我们先计算得到该图片对应类别的score（标量），然后计算该图像每个像素点相对于这个score 的梯度。 如果图像的shape为 `(3, H, W)` ，则此梯度的shape也为 `(3, H, W)` ; 对于图像中的每个像素对应的梯度可以告诉我们，改变这个像素的值，最终对score的影响程度。 为了计算最终的saliency map，我们该梯度的绝对值，然后取3个通道中的最大值。 因此，最终显着性图的形状为 `(H, W)` ，并且所有条目均为非负数。

打开 `daseCV/classifiers/squeezenet.py` 文件并确保你能理解如何使用该模型。你将使用 [`tf.GradientTape()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/GradientTape) 来计算图片每个像素的梯度。 此外，阅读该 [资料](https://www.tensorflow.org/alpha/tutorials/eager/automatic_differentiation#gradient_tapes) 将能促进你的理解。

[2] Karen Simonyan, Andrea Vedaldi, and Andrew Zisserman. "Deep Inside Convolutional Networks: Visualising
Image Classification Models and Saliency Maps", ICLR Workshop 2014.

完成 ```daseCV/net_visualization_tensorflow.py``` 中的 ```compute_saliency_maps``` 函数

### Hint: Tensorflow `gather_nd` method
回想在作业1里面你如何在矩阵的每行选取特定位置的元素。假设 `s` 是一个 `(N, C)` 的numpy array， `y` 是一个 `(N,)` 的numpy array，其中 `0 <= y[i] < C` ，你会用到 `s[np.arange(N), y]` 这样的方法得到一个 `(N,)` 的numpy array，其中的每一行都是用 `y` 作为下标，从 `s` 中选取的数。

在 Tensorflow 中你可以用 `gather_nd()` 函数做同样的事情。 如果 `s` 是一个 `(N, C)` 的 Tensor ， `y` 是一个类型为longs的维度为 `(N,)` 的 Tensor，其中 `0 <= y[i] < C` ，那么 `tf.gather_nd(s, tf.stack((tf.range(N), y), axis=1))` 会得到一个 `(N,)` 的 Tensor，其中的每一行都是用 `y` 作为下标，从 `s` 中选取的数。

你也可以阅读一下文档 [gather_nd method](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/gather_nd).

完成上述任务后，请运行以下命令以可视化来自ImageNet验证集的示例图像上的某些类的saliency maps：

In [ ]:
from daseCV.net_visualization_tensorflow import compute_saliency_maps
def show_saliency_maps(X, y, mask):
    mask = np.asarray(mask)
    Xm = X[mask]
    ym = y[mask]

    saliency = compute_saliency_maps(Xm, ym, model)

    for i in range(mask.size):
        plt.subplot(2, mask.size, i + 1)
        plt.imshow(deprocess_image(Xm[i]))
        plt.axis('off')
        plt.title(class_names[ym[i]])
        plt.subplot(2, mask.size, mask.size + i + 1)
        plt.title(mask[i])
        plt.imshow(saliency[i], cmap=plt.cm.hot)
        plt.axis('off')
        plt.gcf().set_size_inches(10, 4)
    plt.show()

mask = np.arange(5)
show_saliency_maps(X, y, mask)

# INLINE QUESTION

为了最大化正确类别的score，我们可以在输入图像上执行梯度上升的操作，因此我们可以在每个步骤中使用saliency map来代替梯度来更新图像。 这个说法对吗？ 为什么对或者错？

**Your Answer:** 



# Fooling Images
我们也可以使用图像梯度来生成如[3]中所述的 "fooling images" 。给定一个图像和一个目标类别，我们可以对图像执行梯度 **上升** 以最大化目标类别的得分，并在网络将图像分类为目标类别时停止。 实现以下功能以生成fooling images。

[3] Szegedy et al, "Intriguing properties of neural networks", ICLR 2014

完成 ```daseCV/net_visualization_tensforflow.py``` 中的 ```make_fooling_image``` 函数


运行以下单元格以生成一个fooling image。 理想情况下，你应该一眼看不出原始图像和fooling image之间的区别，并且网络现在应该对fooling image做出错误的预测。但是，如果对原始图像和fooling image之间的差异放大10倍，你应该会看到一些随机噪声。 请随时更改 `idx` 变量以浏览其他图像。

In [ ]:
from daseCV.net_visualization_tensorflow import make_fooling_image

idx = 0
Xi = X[idx][None]
target_y = 6
X_fooling = make_fooling_image(Xi, target_y, model)

# Make sure that X_fooling is classified as y_target
scores = model(X_fooling)
assert tf.math.argmax(scores[0]).numpy() == target_y, 'The network is not fooled!'

# Show original image, fooling image, and difference
orig_img = deprocess_image(Xi[0])
fool_img = deprocess_image(X_fooling[0])
plt.figure(figsize=(12, 6))

# Rescale 
plt.subplot(1, 4, 1)
plt.imshow(orig_img)
plt.axis('off')
plt.title(class_names[y[idx]])
plt.subplot(1, 4, 2)
plt.imshow(fool_img)
plt.title(class_names[target_y])
plt.axis('off')
plt.subplot(1, 4, 3)
plt.title('Difference')
plt.imshow(deprocess_image((Xi-X_fooling)[0]))
plt.axis('off')
plt.subplot(1, 4, 4)
plt.title('Magnified difference (10x)')
plt.imshow(deprocess_image(10 * (Xi-X_fooling)[0]))
plt.axis('off')
plt.gcf().tight_layout()

# Class visualization
通过从随机噪声图像开始并对目标类别进行梯度上升，我们可以生成该类别的图像。 这个想法最早是在[2]中提出的。 [3]通过提出几种可以提高所生成图像质量的正则化技术，扩展了这一思想。

具体来说，让 $I$ 作为图像，让 $y$ 作为目标类。对于类 $y$ 卷积网络分配给图像 $I$ 该类的分数为 $s_y(I)$； 请注意，这些是原始的unnormalized的score，而不是类别概率。 我们希望生成一个图像 $I^*$ ，从而使得类 $y$ 获得高分。

$$
I^* = \arg\max_I (s_y(I) - R(I))
$$

$R$ 是一个比较抽象的正则化函数 (请注意argmax中的 $R(I)$ : 我们想最小化这一项). 我们可以通过梯度上升来求解这个优化问题, 计算生成图像的梯度。我们会使用 (显性的) L2 正则化

$$
R(I) = \lambda \|I\|_2^2
$$

**以及** 隐性的 [3] 中介绍的方法来周期性地对生成图像进行模糊处理。通过对生成图片进行梯度上升可以解决这个问题。

[2] Karen Simonyan, Andrea Vedaldi, and Andrew Zisserman. "Deep Inside Convolutional Networks: Visualising
Image Classification Models and Saliency Maps", ICLR Workshop 2014.

[3] Yosinski et al, "Understanding Neural Networks Through Deep Visualization", ICML 2015 Deep Learning Workshop



在 `daseCV/net_visualization_tensorflow.py` 中完成下面的 `create_class_visualization` 函数中用到的 `image_visualization_update_step` 这部分代码。
完成该部分代码后，运行以下单元格以生成狼蛛的图像：

In [ ]:
from daseCV.net_visualization_tensorflow import class_visualization_update_step, jitter, blur_image

def create_class_visualization(target_y, model, **kwargs):
    """
    Generate an image to maximize the score of target_y under a pretrained model.
    
    Inputs:
    - target_y: Integer in the range [0, 1000) giving the index of the class
    - model: A pretrained CNN that will be used to generate the image
    
    Keyword arguments:
    - l2_reg: Strength of L2 regularization on the image
    - learning_rate: How big of a step to take
    - num_iterations: How many iterations to use
    - blur_every: How often to blur the image as an implicit regularizer
    - max_jitter: How much to jitter the image as an implicit regularizer
    - show_every: How often to show the intermediate result
    """
    l2_reg = kwargs.pop('l2_reg', 1e-3)
    learning_rate = kwargs.pop('learning_rate', 25)
    num_iterations = kwargs.pop('num_iterations', 100)
    blur_every = kwargs.pop('blur_every', 10)
    max_jitter = kwargs.pop('max_jitter', 16)
    show_every = kwargs.pop('show_every', 25)
    
    # We use a single image of random noise as a starting point
    X = 255 * np.random.rand(224, 224, 3)
    X = preprocess_image(X)[None]

    loss = None # scalar loss
    grad = None # gradient of loss with respect to model.image, same size as model.image
    
    X = tf.Variable(X)
    for t in range(num_iterations):
        # Randomly jitter the image a bit; this gives slightly nicer results
        ox, oy = np.random.randint(0, max_jitter, 2)
        X = jitter(X, ox, oy)
        X = class_visualization_update_step(X, model, target_y, l2_reg, learning_rate)
        # Undo the jitter
        X = jitter(X, -ox, -oy)
        # As a regularizer, clip and periodically blur
        
        X = tf.clip_by_value(X, -SQUEEZENET_MEAN/SQUEEZENET_STD, (1.0 - SQUEEZENET_MEAN)/SQUEEZENET_STD)
        if t % blur_every == 0:
            X = blur_image(X, sigma=0.5)

        # Periodically show the image
        if t == 0 or (t + 1) % show_every == 0 or t == num_iterations - 1:
            plt.imshow(deprocess_image(X[0]))
            class_name = class_names[target_y]
            plt.title('%s\nIteration %d / %d' % (class_name, t + 1, num_iterations))
            plt.gcf().set_size_inches(4, 4)
            plt.axis('off')
            plt.show()
    return X

Once you have completed the implementation in the cell above, run the following cell to generate an image of Tarantula:

In [ ]:
target_y = 76 # Tarantula
out = create_class_visualization(target_y, model)

在其他类别上尝试你的class visualization！ 可以调整各种超参数来尝试改善所生成图像的质量。（非必须）

In [ ]:
target_y = np.random.randint(1000)
# target_y = 78 # Tick
# target_y = 187 # Yorkshire Terrier
# target_y = 683 # Oboe
# target_y = 366 # Gorilla
# target_y = 604 # Hourglass
print(class_names[target_y])
X = create_class_visualization(target_y, model)